In [1]:
import time
import pandas as pd
from py2neo import Graph, Node, Relationship
from textgenrnn import textgenrnn

Using TensorFlow backend.


In [2]:
def query_to_df(query, graph):
    print("Starting query...", end=" ")
    query_start_time = time.time()
    df = graph.run(query).to_data_frame()
    print("Done ({:.2f} minutes).".format((time.time()-query_start_time)/60))
    return df

In [3]:
graph = Graph( "bolt://matlaber5.media.mit.edu:7687", auth=('neo4j','myneo'))
print("Connected to graph database with {:,} nodes and {:,} relationships!".format(
    graph.database.primitive_counts['NumberOfNodeIdsInUse'], 
    graph.database.primitive_counts['NumberOfRelationshipIdsInUse']))

Connected to graph database with 278,432,359 nodes and 1,844,501,832 relationships!


In [26]:
def gen_query(venue):
    query =  """
    MATCH (a:Author)-[:AUTHORED]-(q:Quanta)
    WHERE q.venue = '{}'
    RETURN COLLECT(q.title) AS titles
    """.format(venue)
    return query

In [27]:
top_5 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science']
top_10 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science', 'Journal of the American Chemical Society', 'JAMA', 'The New England Journal of Medicine', 'Nature Genetics', 'Neuron']
top_42 = ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']

In [41]:
for venue in top_5:
    query = gen_query(venue)
    df = query_to_df(query, graph)
    df.to_csv('C:\\Users\\Brend\\Downloads\\Title Gen\\{}_titles.csv'.format(venue), index = False, encoding = "UTF-8")

Starting query... Done (0.58 minutes).
Starting query... Done (1.60 minutes).
Starting query... Done (0.10 minutes).
Starting query... Done (1.84 minutes).
Starting query... Done (1.58 minutes).


In [97]:
import ast
import random
def gen_titles(venue, num_titles=5, num_samples=1000):
    df_titles = pd.read_csv('C:\\Users\\Brend\\Downloads\\Title Gen\\{}_titles.csv'.format(venue), encoding = "UTF-8")
    title_list = df_titles.iloc[0]['titles']
    tl = ast.literal_eval(title_list)
    print('Venue: ', venue)
    print('Number of titles:', len(tl))
    print('Sampling: ', num_samples)
    texts = []
    for i in range(num_samples):
        texts.append(random.choice(tl))
    textgen = textgenrnn()
    textgen.train_on_texts(texts, num_epochs=1, batch_size=128)
    textgen.generate_to_file('Top5_Generated\\{}_generated.txt'.format(venue),
                    n=num_titles, 
                    temperature=[.2 for i in range(num_titles)],
                    max_gen_length=100,
                    progress=False)

In [98]:
for venue in top_5:
    gen_titles(venue)

Venue:  Cell
Number of titles: 99673
Sampling:  1000
Training on 89,296 character sequences.
Epoch 1/1


314/697 [============>.................] - ETA: 1:31:48 - loss: 2.69 - ETA: 52:53 - loss: 2.6654 - ETA: 39:50 - loss: 2.71 - ETA: 33:18 - loss: 2.75 - ETA: 29:23 - loss: 2.73 - ETA: 26:50 - loss: 2.73 - ETA: 24:58 - loss: 2.76 - ETA: 23:33 - loss: 2.76 - ETA: 22:26 - loss: 2.78 - ETA: 21:32 - loss: 2.77 - ETA: 20:49 - loss: 2.76 - ETA: 20:12 - loss: 2.73 - ETA: 19:40 - loss: 2.74 - ETA: 19:13 - loss: 2.73 - ETA: 18:50 - loss: 2.71 - ETA: 18:29 - loss: 2.68 - ETA: 18:10 - loss: 2.66 - ETA: 17:53 - loss: 2.65 - ETA: 17:38 - loss: 2.64 - ETA: 17:24 - loss: 2.62 - ETA: 17:12 - loss: 2.60 - ETA: 17:01 - loss: 2.59 - ETA: 16:52 - loss: 2.58 - ETA: 16:42 - loss: 2.56 - ETA: 16:32 - loss: 2.54 - ETA: 16:24 - loss: 2.52 - ETA: 16:15 - loss: 2.52 - ETA: 16:08 - loss: 2.50 - ETA: 15:59 - loss: 2.49 - ETA: 15:52 - loss: 2.47 - ETA: 15:46 - loss: 2.47 - ETA: 15:40 - loss: 2.47 - ETA: 15:35 - loss: 2.46 - ETA: 15:29 - loss: 2.44 - ETA: 15:23 - loss: 2.44 - ETA: 15:18 - loss: 2.44 - ETA: 15:14 - loss

628/697 [==========================>...] - ETA: 8:30 - loss: 1.970 - ETA: 8:29 - loss: 1.969 - ETA: 8:28 - loss: 1.968 - ETA: 8:26 - loss: 1.968 - ETA: 8:25 - loss: 1.967 - ETA: 8:24 - loss: 1.967 - ETA: 8:22 - loss: 1.967 - ETA: 8:21 - loss: 1.967 - ETA: 8:20 - loss: 1.966 - ETA: 8:18 - loss: 1.965 - ETA: 8:17 - loss: 1.965 - ETA: 8:16 - loss: 1.965 - ETA: 8:14 - loss: 1.964 - ETA: 8:13 - loss: 1.964 - ETA: 8:12 - loss: 1.963 - ETA: 8:10 - loss: 1.962 - ETA: 8:09 - loss: 1.962 - ETA: 8:08 - loss: 1.962 - ETA: 8:06 - loss: 1.961 - ETA: 8:05 - loss: 1.960 - ETA: 8:04 - loss: 1.960 - ETA: 8:02 - loss: 1.958 - ETA: 8:01 - loss: 1.958 - ETA: 8:00 - loss: 1.958 - ETA: 7:58 - loss: 1.958 - ETA: 7:57 - loss: 1.957 - ETA: 7:56 - loss: 1.956 - ETA: 7:54 - loss: 1.955 - ETA: 7:53 - loss: 1.954 - ETA: 7:52 - loss: 1.955 - ETA: 7:50 - loss: 1.954 - ETA: 7:49 - loss: 1.954 - ETA: 7:48 - loss: 1.954 - ETA: 7:46 - loss: 1.953 - ETA: 7:45 - loss: 1.952 - ETA: 7:44 - loss: 1.952 - ETA: 7:42 - loss: 1.9

697/697 [==============================] - ETA: 1:30 - loss: 1.838 - ETA: 1:29 - loss: 1.838 - ETA: 1:28 - loss: 1.838 - ETA: 1:26 - loss: 1.838 - ETA: 1:25 - loss: 1.838 - ETA: 1:24 - loss: 1.837 - ETA: 1:22 - loss: 1.837 - ETA: 1:21 - loss: 1.837 - ETA: 1:20 - loss: 1.836 - ETA: 1:18 - loss: 1.836 - ETA: 1:17 - loss: 1.835 - ETA: 1:16 - loss: 1.836 - ETA: 1:14 - loss: 1.835 - ETA: 1:13 - loss: 1.835 - ETA: 1:12 - loss: 1.835 - ETA: 1:10 - loss: 1.835 - ETA: 1:09 - loss: 1.834 - ETA: 1:08 - loss: 1.834 - ETA: 1:06 - loss: 1.833 - ETA: 1:05 - loss: 1.832 - ETA: 1:04 - loss: 1.832 - ETA: 1:02 - loss: 1.832 - ETA: 1:01 - loss: 1.832 - ETA: 1:00 - loss: 1.831 - ETA: 58s - loss: 1.831 - ETA: 57s - loss: 1.83 - ETA: 56s - loss: 1.83 - ETA: 54s - loss: 1.83 - ETA: 53s - loss: 1.83 - ETA: 52s - loss: 1.83 - ETA: 50s - loss: 1.83 - ETA: 49s - loss: 1.83 - ETA: 48s - loss: 1.83 - ETA: 46s - loss: 1.83 - ETA: 45s - loss: 1.83 - ETA: 44s - loss: 1.83 - ETA: 42s - loss: 1.83 - ETA: 41s - loss: 1.8

314/485 [==================>...........] - ETA: 1:07:55 - loss: 2.40 - ETA: 39:18 - loss: 2.7294 - ETA: 29:47 - loss: 2.70 - ETA: 25:01 - loss: 2.71 - ETA: 22:08 - loss: 2.71 - ETA: 20:13 - loss: 2.71 - ETA: 18:51 - loss: 2.71 - ETA: 17:49 - loss: 2.71 - ETA: 17:01 - loss: 2.67 - ETA: 16:21 - loss: 2.64 - ETA: 15:49 - loss: 2.63 - ETA: 15:21 - loss: 2.61 - ETA: 14:58 - loss: 2.61 - ETA: 14:39 - loss: 2.60 - ETA: 14:21 - loss: 2.61 - ETA: 14:05 - loss: 2.61 - ETA: 13:51 - loss: 2.60 - ETA: 13:38 - loss: 2.60 - ETA: 13:26 - loss: 2.59 - ETA: 13:16 - loss: 2.58 - ETA: 13:06 - loss: 2.58 - ETA: 12:57 - loss: 2.58 - ETA: 12:49 - loss: 2.57 - ETA: 12:41 - loss: 2.56 - ETA: 12:35 - loss: 2.54 - ETA: 12:28 - loss: 2.53 - ETA: 12:22 - loss: 2.53 - ETA: 12:15 - loss: 2.51 - ETA: 12:10 - loss: 2.51 - ETA: 12:05 - loss: 2.50 - ETA: 11:59 - loss: 2.49 - ETA: 11:55 - loss: 2.50 - ETA: 11:50 - loss: 2.49 - ETA: 11:45 - loss: 2.48 - ETA: 11:41 - loss: 2.49 - ETA: 11:37 - loss: 2.48 - ETA: 11:33 - loss

...

314/790 [==========>...................] - ETA: 1:55:40 - loss: 2.46 - ETA: 1:06:58 - loss: 4.40 - ETA: 50:53 - loss: 3.7768 - ETA: 42:46 - loss: 3.49 - ETA: 37:51 - loss: 3.36 - ETA: 34:33 - loss: 3.25 - ETA: 32:11 - loss: 3.11 - ETA: 30:25 - loss: 3.07 - ETA: 29:02 - loss: 3.02 - ETA: 27:55 - loss: 2.98 - ETA: 27:00 - loss: 2.93 - ETA: 26:14 - loss: 2.89 - ETA: 25:35 - loss: 2.86 - ETA: 25:02 - loss: 2.83 - ETA: 24:32 - loss: 2.80 - ETA: 24:07 - loss: 2.79 - ETA: 23:44 - loss: 2.78 - ETA: 23:23 - loss: 2.77 - ETA: 23:07 - loss: 2.75 - ETA: 22:55 - loss: 2.73 - ETA: 23:00 - loss: 2.72 - ETA: 23:04 - loss: 2.72 - ETA: 23:08 - loss: 2.71 - ETA: 23:01 - loss: 2.69 - ETA: 22:58 - loss: 2.68 - ETA: 22:49 - loss: 2.66 - ETA: 22:41 - loss: 2.65 - ETA: 22:33 - loss: 2.65 - ETA: 22:27 - loss: 2.65 - ETA: 22:19 - loss: 2.64 - ETA: 22:13 - loss: 2.64 - ETA: 22:06 - loss: 2.64 - ETA: 22:00 - loss: 2.63 - ETA: 21:55 - loss: 2.62 - ETA: 21:49 - loss: 2.61 - ETA: 21:44 - loss: 2.60 - ETA: 21:38 - lo

628/790 [======================>.......] - ETA: 13:01 - loss: 2.03 - ETA: 12:59 - loss: 2.03 - ETA: 12:57 - loss: 2.03 - ETA: 12:55 - loss: 2.03 - ETA: 12:54 - loss: 2.03 - ETA: 12:52 - loss: 2.03 - ETA: 12:50 - loss: 2.03 - ETA: 12:49 - loss: 2.03 - ETA: 12:47 - loss: 2.03 - ETA: 12:45 - loss: 2.03 - ETA: 12:44 - loss: 2.03 - ETA: 12:42 - loss: 2.03 - ETA: 12:40 - loss: 2.03 - ETA: 12:39 - loss: 2.03 - ETA: 12:37 - loss: 2.03 - ETA: 12:35 - loss: 2.03 - ETA: 12:34 - loss: 2.03 - ETA: 12:32 - loss: 2.03 - ETA: 12:30 - loss: 2.03 - ETA: 12:29 - loss: 2.03 - ETA: 12:27 - loss: 2.03 - ETA: 12:25 - loss: 2.03 - ETA: 12:24 - loss: 2.02 - ETA: 12:22 - loss: 2.02 - ETA: 12:20 - loss: 2.02 - ETA: 12:18 - loss: 2.02 - ETA: 12:17 - loss: 2.02 - ETA: 12:15 - loss: 2.02 - ETA: 12:13 - loss: 2.02 - ETA: 12:12 - loss: 2.02 - ETA: 12:10 - loss: 2.02 - ETA: 12:08 - loss: 2.02 - ETA: 12:07 - loss: 2.02 - ETA: 12:05 - loss: 2.02 - ETA: 12:03 - loss: 2.02 - ETA: 12:02 - loss: 2.02 - ETA: 12:00 - loss: 2.

790/790 [==============================] - ETA: 4:29 - loss: 1.918 - ETA: 4:27 - loss: 1.917 - ETA: 4:26 - loss: 1.917 - ETA: 4:24 - loss: 1.917 - ETA: 4:22 - loss: 1.916 - ETA: 4:21 - loss: 1.916 - ETA: 4:19 - loss: 1.916 - ETA: 4:17 - loss: 1.916 - ETA: 4:16 - loss: 1.915 - ETA: 4:14 - loss: 1.915 - ETA: 4:12 - loss: 1.914 - ETA: 4:11 - loss: 1.914 - ETA: 4:09 - loss: 1.914 - ETA: 4:07 - loss: 1.914 - ETA: 4:05 - loss: 1.913 - ETA: 4:04 - loss: 1.913 - ETA: 4:02 - loss: 1.913 - ETA: 4:00 - loss: 1.912 - ETA: 3:59 - loss: 1.912 - ETA: 3:57 - loss: 1.912 - ETA: 3:55 - loss: 1.912 - ETA: 3:54 - loss: 1.912 - ETA: 3:52 - loss: 1.912 - ETA: 3:50 - loss: 1.912 - ETA: 3:49 - loss: 1.911 - ETA: 3:47 - loss: 1.911 - ETA: 3:45 - loss: 1.911 - ETA: 3:44 - loss: 1.911 - ETA: 3:42 - loss: 1.910 - ETA: 3:40 - loss: 1.910 - ETA: 3:39 - loss: 1.910 - ETA: 3:37 - loss: 1.909 - ETA: 3:35 - loss: 1.909 - ETA: 3:34 - loss: 1.909 - ETA: 3:32 - loss: 1.909 - ETA: 3:30 - loss: 1.908 - ETA: 3:29 - loss: 1.9

314/517 [=================>............] - ETA: 1:17:55 - loss: 1.99 - ETA: 45:07 - loss: 2.2629 - ETA: 34:34 - loss: 2.38 - ETA: 29:11 - loss: 2.46 - ETA: 25:47 - loss: 2.52 - ETA: 23:26 - loss: 2.56 - ETA: 21:45 - loss: 2.57 - ETA: 20:29 - loss: 2.57 - ETA: 19:30 - loss: 2.54 - ETA: 18:42 - loss: 2.56 - ETA: 18:02 - loss: 2.59 - ETA: 17:30 - loss: 2.58 - ETA: 17:02 - loss: 2.57 - ETA: 16:37 - loss: 2.56 - ETA: 16:16 - loss: 2.54 - ETA: 15:56 - loss: 2.53 - ETA: 15:40 - loss: 2.53 - ETA: 15:25 - loss: 2.53 - ETA: 15:11 - loss: 2.51 - ETA: 14:59 - loss: 2.51 - ETA: 14:48 - loss: 2.51 - ETA: 14:38 - loss: 2.49 - ETA: 14:28 - loss: 2.48 - ETA: 14:19 - loss: 2.48 - ETA: 14:10 - loss: 2.48 - ETA: 14:02 - loss: 2.48 - ETA: 13:55 - loss: 2.47 - ETA: 13:48 - loss: 2.46 - ETA: 13:42 - loss: 2.45 - ETA: 13:35 - loss: 2.44 - ETA: 13:30 - loss: 2.44 - ETA: 13:24 - loss: 2.43 - ETA: 13:19 - loss: 2.42 - ETA: 13:14 - loss: 2.41 - ETA: 13:09 - loss: 2.41 - ETA: 13:05 - loss: 2.41 - ETA: 13:00 - loss

517/517 [==============================] - ETA: 5:06 - loss: 2.086 - ETA: 5:05 - loss: 2.085 - ETA: 5:03 - loss: 2.084 - ETA: 5:02 - loss: 2.084 - ETA: 5:00 - loss: 2.084 - ETA: 4:59 - loss: 2.084 - ETA: 4:57 - loss: 2.084 - ETA: 4:56 - loss: 2.083 - ETA: 4:55 - loss: 2.083 - ETA: 4:53 - loss: 2.083 - ETA: 4:52 - loss: 2.083 - ETA: 4:50 - loss: 2.082 - ETA: 4:49 - loss: 2.081 - ETA: 4:47 - loss: 2.081 - ETA: 4:46 - loss: 2.081 - ETA: 4:44 - loss: 2.081 - ETA: 4:43 - loss: 2.080 - ETA: 4:41 - loss: 2.080 - ETA: 4:40 - loss: 2.080 - ETA: 4:38 - loss: 2.080 - ETA: 4:37 - loss: 2.080 - ETA: 4:35 - loss: 2.079 - ETA: 4:34 - loss: 2.078 - ETA: 4:33 - loss: 2.078 - ETA: 4:31 - loss: 2.078 - ETA: 4:30 - loss: 2.077 - ETA: 4:28 - loss: 2.076 - ETA: 4:27 - loss: 2.076 - ETA: 4:25 - loss: 2.076 - ETA: 4:24 - loss: 2.075 - ETA: 4:22 - loss: 2.074 - ETA: 4:21 - loss: 2.073 - ETA: 4:19 - loss: 2.073 - ETA: 4:18 - loss: 2.073 - ETA: 4:16 - loss: 2.072 - ETA: 4:15 - loss: 2.071 - ETA: 4:13 - loss: 2.0